In [2]:
import torch, struct, os, psutil, subprocess, time, threading
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import tensorflow as tf

2024-01-31 20:16:08.413462: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-31 20:16:08.433309: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 20:16:08.433333: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 20:16:08.433860: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 20:16:08.437325: I tensorflow/core/platform/cpu_feature_guar

### Load and Evaluate

In [3]:
# TODO: use json file for model (results, config etc) management
model_name = "dnn_mnist_pt"
model_path = "../../models/"

state_dict = torch.load(model_path + model_name + ".pth", map_location=torch.device("cpu"))
list_vars = state_dict

# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Convert to PyTorch tensors
test_images_pt = torch.tensor(test_images).float()
# Flatten and normalize the images
test_images_pt = test_images_pt.view(-1, 28*28) / 255.0  # Flatten and normalize


In [4]:
class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 56)  # Flatten 28*28 and feed into 56 neurons
        self.fc2 = nn.Linear(56, num_classes)  # 56 inputs, 10 outputs (number of classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1)
    
model_pt = Net()
model_pt.load_state_dict(state_dict)
model_pt.eval()  # Set the model to evaluation mode

with torch.no_grad():  # Ensure gradients are not computed
    predictions = model_pt(test_images_pt)
    predicted_labels = predictions.argmax(dim=1)
predictions = predicted_labels.tolist()

### Setup

In [5]:
folder = "./tmp/"

# Create the directory 'tmp' in the current working directory
try:
    os.makedirs(folder, exist_ok=True)
    print(f"Directory '{folder}' created successfully")
except OSError as error:
    print(f"Directory '{folder}' cannot be created. Error: {error}")

Directory './tmp/' created successfully


In [6]:
fname_out = folder + "ggml-model-" + model_name + ".bin"
pack_fmt = "!i"

fout = open(fname_out, "w+b")
fout.write(struct.pack(pack_fmt, 0x67676d6c)) # magic: ggml in hex

for name in list_vars.keys():
    data = list_vars[name].squeeze().numpy()
    print("Processing variable: " + name + " with shape: ", data.shape) 
    n_dims = len(data.shape)
   
    fout.write(struct.pack(pack_fmt, n_dims))
    
    data = data.astype(np.float32)
    for i in range(n_dims):
        fout.write(struct.pack(pack_fmt, data.shape[n_dims - 1 - i]))

    # data
    data = data.astype(">f4")
    data.tofile(fout)

fout.close()

Processing variable: fc1.weight with shape:  (56, 784)
Processing variable: fc1.bias with shape:  (56,)
Processing variable: fc2.weight with shape:  (10, 56)
Processing variable: fc2.bias with shape:  (10,)


In [7]:
def load_img_from_file(data_file="input", show=False):
    try:
        with open(data_file, 'rb') as file:
            buf = file.read()
    except Exception as e:
        print(e)
        return None, e

    digits = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
    
    if show:
        c = ""
        for row in range(28):
            for col in range(28):
                if buf[row * 28 + col] > 230:
                    c += "&"
                else:
                    c += "-"
            c += "\n"
        print(c)

    return digits, None

def save_img_to_file(image, data_file = "input"):
    try:
        # Convert to bytes
        image_bytes = image.astype('uint8').tobytes()
        
        # Write to file
        with open(data_file, 'wb') as file:
            file.write(image_bytes)
        
        #print(f"Image saved to {data_file}")
    except Exception as e:
        print(f"Error saving image: {e}")


In [21]:
def monitor_memory(pid, freq = 0.01):
    p = psutil.Process(pid)
    max_memory = 0
    while True:
        try:
            mem = p.memory_info().rss / (1024 * 1024)
            max_memory = max(max_memory, mem)
        except psutil.NoSuchProcess:
            break  # Process has finished
        time.sleep(freq)  # Poll every second
        
    #print(f"Maximum memory used: {max_memory} MB")
    return max_memory

### Test

In [13]:
ind = 8
img_out_path = folder + str(ind)
save_img_to_file(test_images[ind], img_out_path)

In [14]:
vm_file = "./bin/vm"
program = "./bin/gglm.bin"
model_in_path = fname_out

command = [f"{vm_file}", f"--basedir={folder}",
                f"--program={program}", f"--model={model_in_path}", 
                f"--data={img_out_path}", "--mipsVMCompatible"]

In [15]:
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Get the process ID
pid = process.pid
print(f"Process ID: {pid}")

# Start memory monitoring in a separate thread
monitor_thread = threading.Thread(target=monitor_memory, args=(pid,))
monitor_thread.start()

# Wait for the process to complete and capture output
stdout, stderr = process.communicate()

# Wait for the monitoring thread to finish
monitor_thread.join()

stderr[-2]

Process ID: 487774
Maximum memory used: 86.6953125 MB


'6'

In [16]:
# Example usage
digits, error = load_img_from_file(img_out_path,show=True)

----------------------------
----------------------------
----------------------------
----------------------------
----------------------------
------------------&&&-&&&---
--------------&&&&&&&&&&&---
-------------&&&&&&&--------
-------------&-&------------
---------&------------------
--------&-------------------
-------&&-------------------
------&---------------------
------&---------------------
-----&&&--------------------
-----&&&&&------------------
------&&&&&&&&&&&&----------
---------&&&&&&&&&&---------
------------&&&&&&&---------
------------&&---&&---------
------------&&&-&&&---------
-------------&&&&&----------
--------------&&------------
----------------------------
----------------------------
----------------------------
----------------------------
----------------------------



### Benchmark

In [22]:
import concurrent.futures
def benchmark(test_images, test_labels, tmp_folder, model_in_path, vm_file = "./bin/vm", program = "./bin/gglm.bin", threaded = True):
    benchmark_start_time = time.time()
    mem = []
    results = []
    for ind, img in enumerate(test_images):
        img_out_path = tmp_folder + str(ind)
        save_img_to_file(img, img_out_path)

        # Exclusion of Pre-processing
        start_time = time.time()
        command = [f"{vm_file}", f"--basedir={folder}",
                f"--program={program}", f"--model={model_in_path}", 
                f"--data={img_out_path}", "--mipsVMCompatible"]
        
        print ("Process for image", ind)
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        # Get the process ID
        pid = process.pid

        if threaded:
            with concurrent.futures.ThreadPoolExecutor() as executor:
                future = executor.submit(monitor_memory, pid)
                _, stderr = process.communicate()
                max_memory = future.result()
                #print(f"Maximum memory used in multi-threaded mode: {max_memory} bytes")
        else:
            max_memory = monitor_memory(pid)  # Run in the same thread
            _, stderr = process.communicate()
            #print(f"Maximum memory used in single-threaded mode: {max_memory} bytes")
        mem.append(max_memory)
        #print ("Ind:", ind, "mem:", max_memory, "time:", time.time() - start_time)
        results.append([int(stderr[-2]), max_memory, time.time() - start_time])
    
    print ("Total time:", time.time() - benchmark_start_time)
    #print ("total mem:", sum(mem))
    return results

In [23]:
results = benchmark(test_images[:100], test_labels, './tmp/', model_in_path)

Process for image 0
Process for image 1
Process for image 2
Process for image 3
Process for image 4
Process for image 5
Process for image 6
Process for image 7
Process for image 8
Process for image 9
Process for image 10
Process for image 11
Process for image 12
Process for image 13
Process for image 14
Process for image 15
Process for image 16
Process for image 17
Process for image 18
Process for image 19
Process for image 20
Process for image 21
Process for image 22
Process for image 23
Process for image 24
Process for image 25
Process for image 26
Process for image 27
Process for image 28
Process for image 29
Process for image 30
Process for image 31
Process for image 32
Process for image 33
Process for image 34
Process for image 35
Process for image 36
Process for image 37
Process for image 38
Process for image 39
Process for image 40
Process for image 41
Process for image 42
Process for image 43
Process for image 44
Process for image 45
Process for image 46
Process for image 47
Pr

### Save Results

In [24]:
results

[[7, 88.35546875, 3.368110179901123],
 [2, 90.609375, 3.530409336090088],
 [1, 86.88671875, 3.2679519653320312],
 [0, 86.73046875, 3.619429111480713],
 [4, 88.60546875, 3.4474661350250244],
 [1, 86.9453125, 3.3622653484344482],
 [4, 88.25, 3.4326109886169434],
 [9, 87.94140625, 3.520963430404663],
 [6, 89.5078125, 3.4767587184906006],
 [9, 88.52734375, 3.4311928749084473],
 [0, 92.83984375, 3.568319082260132],
 [6, 87.61328125, 3.5399277210235596],
 [9, 92.84765625, 3.420233964920044],
 [0, 88.3359375, 4.561923265457153],
 [1, 87.609375, 3.2792582511901855],
 [5, 86.45703125, 3.5403053760528564],
 [9, 88.68359375, 3.5294783115386963],
 [7, 86.39453125, 3.408545970916748],
 [3, 88.41796875, 3.671759843826294],
 [4, 94.10546875, 3.428042411804199],
 [9, 88.31640625, 3.4201433658599854],
 [6, 94.33984375, 4.49481463432312],
 [6, 86.6640625, 3.4981234073638916],
 [5, 87.7578125, 3.3555214405059814],
 [4, 88.6328125, 3.366218328475952],
 [0, 88.65625, 3.7094602584838867],
 [7, 88.9609375, 3

In [26]:
for i in range(100):
    if predictions[i] != results[i][0]:
        print ("Index", i, "Not match!")

Index 8 Not match!
